# Speed benchmark of five aligners for different thread counts

This benchmark is testing the alignment speed of 5 aligners and pseudo-aligners. Pseudo-aligners are using k-mers (shorter nucleotide fragments) of length 31 in this case. Through an expectation maximization the k-mers from a read are used to predict the most likely transcript from which they originate. This notebook is ment to be executed from the docker image maayanlab/alignmentbenchmark. All dependencies should be installed.

### Pseudo-aligners
- kallisto
- Salmon

### Aligners
- STAR
- HISAT2
- BWA

STAR requires a fairly large amount of memory to run compared to all other methods. The index was built using a spacing of 2 (--genomeSAsparseD 2) which technically halves the memory required from around 30GB to 16GB. As a trade-off alignment is slowed down by 25% but results in the identical gene counts. This setting is more cost efficient in most cloud based applications.

## Requirements

- 26GB of memory
- 100GB of free disk space
- CPU with high thread count (16)

In [ ]:
cd /alignment

# create folders
mkdir -p sradata
mkdir -p quant/combined
mkdir -p quant/salmon/
mkdir -p quant/kallisto/
mkdir -p quant/star/
mkdir -p quant/hisat2/
mkdir -p time

# when using high thread count in STAR there will be a large number of open files. This can lead to errors
# set ulimit -n 1024 and it should work with many threads
ulimit -n 1024

# before we start downloading data we should disable the caching of SRA files
# when downloading many files this will quickly end up filling up all disk space otherwise
# this command below magically works to disable caching
mkdir -p ~/.ncbi
echo '/repository/user/cache-disabled = "true"' > ~/.ncbi/user-settings.mkfg

Select the species that you want the SRA samples to be aligned against and the location of precomputed index files. From "mssm-genecount-combined" species "human" and "mouse" can be retrieved. There is a Jupyter notebook to build new index files.

In [ ]:
SPECIES="human"
AWS_BUCKET="mssm-genecount-combined"

Download pre-computed index files for human. Precomputed mouse index files are also available.

In [ ]:
scripts/load_index_species.sh ${SPECIES} ${AWS_BUCKET}

There are 4 helper functions. "center" just displays the current state of alignment, "downloadSRA" will use faster-dump to download and extract SRA files to fastq. If the sample is based on paired reads it will create two fastq files. "alignPairedSRA" and "alignSingleSRA" will align the previously downloaded fastq file to the aligners Salmon, kallisto, STAR, HISAT2, and BWA.

The resulting quantification counts are then aggregated by an R script into a tsv file for overlapping genes, with columns representing the 5 aligners.

In [ ]:
center() {
  echo "---------------------------------- ${1} ----------------------------------"
}

#small helper function for downloads
downloadSRA(){
    center "Download SRA file"
    rm -f sradata/*
    fasterq-dump \
        --split-files \
        --outdir $2 \
        -e $thread_num -t "sradata" \
        $1
}

alignPairedSRA(){
    SRA_FILE=$1
    SPECIES=$2
    
    center "Salmon"
    
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo salmon >> time/${SRA_FILE}_time.txt
        salmon quant -i "index/salmon/salmon_${SPECIES}_96" -l A \
            -1 "sradata/${SRA_FILE}_1.fastq" \
            -2 "sradata/${SRA_FILE}_2.fastq" \
            -p $thread_num -q --validateMappings -o quant/salmon/$SRA_FILE
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "kallisto"
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo kallisto >> time/${SRA_FILE}_time.txt
        kallisto quant -i index/kallisto/kallisto_${SPECIES}_96.idx \
            "sradata/${SRA_FILE}_1.fastq" \
            "sradata/${SRA_FILE}_2.fastq" \
            -t $thread_num -o quant/kallisto/$SRA_FILE
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "STAR"
    mkdir -p quant/star/$SRA_FILE
    
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo star >> time/${SRA_FILE}_time.txt
        STAR \
            --genomeDir "index/star/${SPECIES}_96" \
            --limitBAMsortRAM 10000000000 \
            --runThreadN $thread_num \
            --outSAMstrandField intronMotif \
            --outFilterIntronMotifs RemoveNoncanonical \
            --outFileNamePrefix quant/star/$SRA_FILE/$SRA_FILE \
            --readFilesIn "sradata/${SRA_FILE}_1.fastq" "sradata/${SRA_FILE}_2.fastq" \
            --outSAMtype BAM SortedByCoordinate \
            --outReadsUnmapped Fastx \
            --outSAMmode Full \
            --quantMode GeneCounts \
            --limitIObufferSize 50000000
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "HISAT2"
    mkdir -p quant/hisat2/$SRA_FILE
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo hisat2 >> time/${SRA_FILE}_time.txt
        hisat2 \
            -x "index/hisat2/${SPECIES}_96/${SPECIES}" \
            -1 "sradata/${SRA_FILE}_1.fastq" \
            -2 "sradata/${SRA_FILE}_2.fastq" \
            -p $thread_num \
            -S "quant/hisat2/${SRA_FILE}/${SRA_FILE}.sam"
        featureCounts -T $thread_num -a "reference/${SPECIES}_96.gtf" -o "quant/hisat2/${SRA_FILE}/${SRA_FILE}.tsv" "quant/hisat2/${SRA_FILE}/${SRA_FILE}.sam"
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "BWA"
    mkdir -p quant/bwa/$SRA_FILE
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo bwa >> time/${SRA_FILE}_time.txt
        bwa mem \
            -t $thread_num \
            "index/bwa/${SPECIES}_96/${SPECIES}_96" \
            "sradata/${SRA_FILE}_1.fastq" \
            "sradata/${SRA_FILE}_2.fastq" \
            > "quant/bwa/${SRA_FILE}/${SRA_FILE}.sam"
        featureCounts -T $thread_num -a "reference/${SPECIES}_96.gtf" -o "quant/bwa/${SRA_FILE}/${SRA_FILE}.tsv" "quant/bwa/${SRA_FILE}/${SRA_FILE}.sam"
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    Rscript --vanilla scripts/aggregatecounts.r $SRA_FILE $SPECIES
}

alignSingleSRA(){
    SRA_FILE=$1
    SPECIES=$2
    
    center "Salmon"
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo Salmon >> time/${SRA_FILE}_time.txt
        salmon quant -i "index/salmon/salmon_${SPECIES}_96" -l A \
            -r "sradata/${SRA_FILE}.fastq" \
            -p $thread_num -q --validateMappings -o quant/salmon/$SRA_FILE
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "kallisto"
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo kallisto >> time/${SRA_FILE}_time.txt
        kallisto quant -i index/kallisto/kallisto_${SPECIES}_96.idx \
            -t $thread_num -o quant/kallisto/$SRA_FILE \
            --single -l 180 -s 20 "sradata/${SRA_FILE}.fastq"
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "STAR"
    mkdir -p quant/star/$SRA_FILE
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo STAR >> time/${SRA_FILE}_time.txt
        STAR \
            --genomeDir "index/star/${SPECIES}_96" \
            --limitBAMsortRAM 10000000000 \
            --runThreadN $thread_num \
            --outSAMstrandField intronMotif \
            --outFilterIntronMotifs RemoveNoncanonical \
            --outFileNamePrefix quant/star/$SRA_FILE/$SRA_FILE \
            --readFilesIn "sradata/${SRA_FILE}.fastq" \
            --outSAMtype BAM SortedByCoordinate \
            --outReadsUnmapped Fastx \
            --outSAMmode Full \
            --quantMode GeneCounts \
            --limitIObufferSize 50000000
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    center "HISAT2"
    mkdir -p quant/hisat2/$SRA_FILE
    { time {
        echo "-----------" >> time/${SRA_FILE}_time.txt
        echo hisat2 >> time/${SRA_FILE}_time.txt
        hisat2 \
            -x "index/hisat2/${SPECIES}_96/${SPECIES}" \
            -U "sradata/${SRA_FILE}.fastq" \
            -p $thread_num \
            -S "quant/hisat2/${SRA_FILE}/${SRA_FILE}.sam"
        featureCounts -T thread_num -a "reference/${SPECIES}_96.gtf" -o "quant/hisat2/${SRA_FILE}/${SRA_FILE}.tsv" "quant/hisat2/${SRA_FILE}/${SRA_FILE}.sam"
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    mkdir -p quant/bwa/$SRA_FILE
    { time {
        bwa mem \
            -t $thread_num \
            "index/bwa/${SPECIES}_96/${SPECIES}_96" \
            "sradata/${SRA_FILE}.fastq" \
            > "quant/bwa/${SRA_FILE}/${SRA_FILE}.sam"
        featureCounts -T $thread_num -a "reference/${SPECIES}_96.gtf" -o "quant/bwa/${SRA_FILE}/${SRA_FILE}.tsv" "quant/bwa/${SRA_FILE}/${SRA_FILE}.sam"
    } 2> temp.txt ; } 2>> time/${SRA_FILE}_time.txt
    
    Rscript --vanilla scripts/aggregatecounts.r $SRA_FILE $SPECIES
}

The file we use for benchmarking the speed of the aligners is SRR2972202 from the sample GSM1962466. The origin of the file is human in vivo single Tcell sequences with Illumina HiSeq 2000 for paired-end reads. The file contains 3,161,833 spots, which is on the smaller size. When using other files with significantly higher read it should be kept in mind that some of the algorithms will take fairly long to finish when using low thread counts.

https://www.ncbi.nlm.nih.gov/sra?term=SRX1461572

Reinius, Björn, et al. "Analysis of allelic expression patterns in clonal somatic cells by single-cell RNA–seq." Nature genetics 48.11 (2016): 1430.

In [ ]:
SRA_FILE="SRR2972202"
time downloadSRA $SRA_FILE "sradata"

The benchmark will caluclate the alignment for multiple settings of thread count. When choosing high thread counts make sure the system running the benchmark supports the number of threads selected.

The time to execute each alignment step will be recorded in a text file and saved under the SRA id in the time folder.

In [ ]:
rm -f time/${SRA_FILE}_time.txt
echo "---- file ${SRA_FILE} -------">> time/${SRA_FILE}_time.txt
for i in 1 2 3 4 5 6 8 12 16
do
    thread_num=$i
    echo "---- thread count $i -------">> time/${SRA_FILE}_time.txt
    time alignPairedSRA $SRA_FILE human
done
